# **Task 1: Airline Passenger Data Preprocessing**

**Problem Statement:**
An airline collects passenger booking and check-in data from three sources:

1. **Online Booking System (JSON):** Direct bookings via website/app.
2. **Third-Party Travel Agency (XML):** External agency bookings.
3. **Airport Check-In System (JSON):** Actual check-in records.

Since no single source has complete information, merging them requires handling missing values, inconsistencies, and duplicates.

---

**Dataset Issues & Preprocessing Approach**

### Handling Missing Values
- **Missing Ticket Class (Third-Party Travel Agency):** Fill with "Unknown."
- **Missing Seat Number (Online Booking):** Fill with "Unassigned."
- **Missing Ticket Price (Airport Check-In):** Estimate based on ticket class.
- **Missing Airport Information:** Fill with "Unknown."
- **Missing Payment Status:** Set to "Completed" for online bookings and "Unknown" for check-ins.

### Data Normalization
- **Date & Time:** Convert to YYYY-MM-DD HH:MM:SS UTC format.
- **Phone Numbers:** Standardize format to +CountryCode-XXX-XXX-XXXX.

### Final Preprocessing Steps
1. Extract relevant fields from each dataset.
2. Normalize date-time formats, phone numbers, and ticket class values.
3. Remove duplicate records, keeping the most complete entries.
4. Save the cleaned data as a structured CSV file.

---

**Final CSV Format**

| Column | Description |
|--------|------------|
| booking_id | Unique booking identifier |
| passenger_name | Full name of passenger |
| passport_number | Passport ID |
| email | Contact email |
| phone_number | Standardized phone number |
| flight_number | Flight identifier |
| departure_airport | Departure airport code |
| departure_time_utc | Standardized UTC departure time |
| arrival_airport | Arrival airport code |
| arrival_time_utc | Standardized UTC arrival time |
| ticket_class | Economy, Business, First, or Unknown |
| seat_number | Assigned or "Unassigned" |
| ticket_price_usd | Ticket price or 0.00 if unknown |
| payment_status | Completed, Pending, or Unknown |



In [3]:
import pandas as pd
import json
import xml.etree.ElementTree as ET
from datetime import datetime
import re

def load_json(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        return json.load(file)

def load_xml(filename):
    tree = ET.parse(filename)
    root = tree.getroot()
    data = []
    for record in root.findall("record"):
        entry = {child.tag: child.text for child in record}
        data.append(entry)
    return data

def normalize_phone_number(phone):
    if phone and isinstance(phone, str):
        phone = re.sub(r"[^0-9+]", "", phone)
        return re.sub(r"(\d{3})(\d{3})(\d{4})", r"\1-\2-\3", phone)
    return "Unknown"

def normalize_datetime(date_str):
    formats = ["%Y-%m-%d %H:%M:%S", "%d/%m/%Y %I:%M %p", "%m-%d-%Y %H:%M"]
    for fmt in formats:
        try:
            return datetime.strptime(date_str, fmt).strftime("%Y-%m-%d %H:%M:%S UTC")
        except (ValueError, TypeError):
            continue
    return "Unknown"

def estimate_ticket_price(ticket_class):
    prices = {"Economy": 200.00, "Business": 500.00, "First": 1000.00, "Unknown": 0.00}
    return prices.get(ticket_class, 0.00)

def preprocess_data():
    online_booking = pd.DataFrame(load_json("online_booking.json"))
    travel_agency = pd.DataFrame(load_xml("third_party_travel_agency.xml"))
    airport_checkin = pd.DataFrame(load_json("airport_check_in_data.json"))

    # Process online booking data
    online_booking = online_booking.assign(
        passenger_name=online_booking["first_name"] + " " + online_booking["last_name"],
        email=online_booking.get("contact_email", pd.Series()).fillna("Unknown"),
        phone_number=online_booking.get("contact_phone", pd.Series()).apply(normalize_phone_number),
        ticket_class=online_booking.get("ticket_class", pd.Series()).fillna("Unknown"),
        seat_number=online_booking.get("seat_number", pd.Series()).fillna("Unassigned"),
        departure_airport=online_booking.get("departure_airport", pd.Series()).fillna("Unknown"),
        arrival_airport=online_booking.get("arrival_airport", pd.Series()).fillna("Unknown"),
        departure_time_utc=online_booking.get("departure_time", pd.Series()).apply(normalize_datetime),
        arrival_time_utc=online_booking.get("arrival_time", pd.Series()).apply(normalize_datetime),
        payment_status="Completed"
    )

    # Process travel agency data
    travel_agency = travel_agency.assign(
        passenger_name=travel_agency.get("passenger_name", pd.Series()).fillna("Unknown"),
        email=travel_agency.get("email", pd.Series()).fillna("Unknown"),
        phone_number=travel_agency.get("phone_number", pd.Series()).apply(normalize_phone_number),
        ticket_class=travel_agency.get("ticket_class", pd.Series()).fillna("Unknown"),
        seat_number=travel_agency.get("seat_number", pd.Series()).fillna("Unassigned"),
        departure_airport=travel_agency.get("departure_airport", pd.Series()).fillna("Unknown"),
        arrival_airport=travel_agency.get("arrival_airport", pd.Series()).fillna("Unknown"),
        departure_time_utc=travel_agency.get("departure_time", pd.Series()).apply(normalize_datetime),
        arrival_time_utc=travel_agency.get("arrival_time", pd.Series()).apply(normalize_datetime),
        payment_status=travel_agency.get("payment_status", pd.Series()).fillna("Unknown")
    )

    # Process airport check-in data
    airport_checkin = airport_checkin.assign(
        passenger_name=airport_checkin.get("passenger_name", pd.Series()).fillna("Unknown"),
        email=airport_checkin.get("email", pd.Series()).fillna("Unknown"),
        phone_number=airport_checkin.get("phone_number", pd.Series()).apply(normalize_phone_number),
        ticket_class=airport_checkin.get("ticket_class", pd.Series()).fillna("Unknown"),
        seat_number=airport_checkin.get("seat_number", pd.Series()).fillna("Unassigned"),
        departure_airport=airport_checkin.get("departure_airport", pd.Series()).fillna("Unknown"),
        arrival_airport=airport_checkin.get("arrival_airport", pd.Series()).fillna("Unknown"),
        departure_time_utc=airport_checkin.get("departure_time", pd.Series()).apply(normalize_datetime),
        arrival_time_utc=airport_checkin.get("arrival_time", pd.Series()).apply(normalize_datetime),
        ticket_price_usd=pd.to_numeric(airport_checkin.get("ticket_price_usd", pd.Series()), errors="coerce").fillna(0.00),
        payment_status=airport_checkin.get("payment_status", pd.Series()).fillna("Unknown")
    )

    # Merge all data
    all_data = pd.concat([online_booking, travel_agency, airport_checkin], ignore_index=True)

    # Ensure ticket price is set
    all_data["ticket_price_usd"] = all_data.apply(
        lambda row: row["ticket_price_usd"] if row["ticket_price_usd"] else estimate_ticket_price(row["ticket_class"]),
        axis=1
    )

    # Ensure required columns exist
    all_data = all_data.assign(
        booking_id=all_data.get("booking_id", pd.Series()).fillna("Unknown"),
        flight_number=all_data.get("flight_number", pd.Series()).fillna("Unknown")
    )

    # Remove duplicates based on unique identifiers
    all_data.drop_duplicates(subset=["booking_id", "passport_number"], keep="first", inplace=True)

    # Define final column order
    final_columns = [
        "booking_id", "passenger_name", "passport_number", "email", "phone_number",
        "flight_number", "departure_airport", "departure_time_utc",
        "arrival_airport", "arrival_time_utc", "ticket_class", "seat_number",
        "ticket_price_usd", "payment_status"
    ]

    # Select and reorder columns
    all_data = all_data[[col for col in final_columns if col in all_data.columns]]

    # Save to CSV
    all_data.to_csv("cleaned_passenger_data.csv", index=False)
    print("Data preprocessing complete. CSV file saved.")

preprocess_data()


Data preprocessing complete. CSV file saved.


# **Task 2: Mean, Median, and Mode Analysis**

**Problem Statement:**
This task explores the effectiveness of mean, median, and mode in different datasets. The goal is to analyze which measure provides the most representative value.

---

**Datasets Used:**
- **Income Distribution Dataset (100+ data points):** Includes high-income outliers.
- **Product Rating Dataset (50+ ratings):** Discrete values (1-5) with a clear mode.
- **Temperature Dataset (30+ days):** Normally distributed with minor outliers.

---

**Analysis Steps:**
For each dataset:
1. Calculate Mean, Median, and Mode.
2. Determine which measure best represents the dataset.
3. Explain why that measure is most suitable.
4. Provide a real-world example where using the wrong measure could mislead analysis.

---





In [4]:
import numpy as np
import random
import statistics
import scipy.stats as stats

# 1. Income Distribution Dataset (Array 1)
income_data = np.random.normal(50000, 15000, 95).tolist()  # Normal distribution
income_data += [200000, 220000, 250000, 270000, 300000]  # Adding outliers

# 2. Product Rating Dataset (Array 2)
rating_data = random.choices([1, 2, 3, 4, 5], weights=[5, 10, 25, 40, 20], k=50)  # Biased towards 4-star ratings

# 3. Temperature Dataset (Array 3)
temperature_data = np.random.normal(75, 5, 28).tolist()  # Normal distribution around 75°F
temperature_data += [60, 95, 98]  # Adding minor outliers

# Function to calculate and display statistics
def analyze_dataset(dataset, name):
    mean_value = np.mean(dataset)
    median_value = np.median(dataset)
    mode_value = statistics.mode(dataset)

    print(f"\n{name} Analysis:")
    print(f"Mean: {mean_value:.2f}")
    print(f"Median: {median_value:.2f}")
    print(f"Mode: {mode_value}")

    # Analysis
    if abs(mean_value - median_value) > abs(mode_value - median_value):
        print("Median is more representative due to skewness or outliers.")
    elif mode_value == median_value:
        print("Mode is a good representation, indicating frequent values.")
    else:
        print("Mean is effective when data is normally distributed.")

# Analyzing datasets
analyze_dataset(income_data, "Income Distribution")
analyze_dataset(rating_data, "Product Ratings")
analyze_dataset(temperature_data, "Temperature Data")



Income Distribution Analysis:
Mean: 59941.39
Median: 51993.78
Mode: 63152.85074013027
Mean is effective when data is normally distributed.

Product Ratings Analysis:
Mean: 3.64
Median: 4.00
Mode: 4
Median is more representative due to skewness or outliers.

Temperature Data Analysis:
Mean: 75.12
Median: 73.68
Mode: 71.82099055603614
Mean is effective when data is normally distributed.
